#### 1. Creating Initial Variables

In [10]:
# Create Sagemaker execution role
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
import boto3
import os

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()

s3_prefix = "script-mode-workflow"
pickle_s3_prefix = f"{s3_prefix}/pickle"
pickle_s3_uri = f"s3://{bucket}/{s3_prefix}/pickle"
pickle_train_s3_uri = f"{pickle_s3_uri}/train"

train_dir = os.path.join(os.getcwd(), "")

#### 2. Uploading Training Data to S3

In [11]:
s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_bucket.Object(os.path.join(pickle_s3_prefix, "train.pickle")).upload_file(
    train_dir + "/train.pickle"
)

#### 3. Creating Hyperparameters

In [12]:
hyperparameters = {
    "copy_X": True,
    "fit_intercept": True,
    "normalize": False,
}

#### 4. Creating Estimator Parameters

In [ ]:
entry_point = "regression-script.py"

# Modify this based on your instance type / size
train_instance_type = "ml.m5.large"

estimator_parameters = {
    "entry_point": entry_point,
    "source_dir": "scripts",
    "framework_version": "0.23-1",
    "py_version": "py3",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": hyperparameters,
    "role": role,
    "base_job_name": "linearregression-model",
}

estimator = SKLearn(**estimator_parameters)

#### 5. Training

In [16]:
inputs = {
    "train": pickle_train_s3_uri
}

# starting the training job
estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: linearregression-model-2024-11-17-01-21-11-189


2024-11-17 01:21:14 Starting - Starting the training job...
2024-11-17 01:21:28 Starting - Preparing the instances for training...
2024-11-17 01:22:15 Downloading - Downloading the training image.....2024-11-17 01:23:00,377 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-11-17 01:23:00,380 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-17 01:23:00,422 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-11-17 01:23:00,589 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-17 01:23:00,602 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-17 01:23:00,614 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-17 01:23:00,623 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_di

#### 6. Deploying the Model to an Endpoint

In [17]:
sklearn_predictor = estimator.deploy(initial_instance_count=1,
                                     instance_type='ml.m5.large',
                                     endpoint_name='linearregression-endpoint')

sklearn_predictor.predict([[0],[1],[2],[3]])

INFO:sagemaker:Creating model with name: linearregression-model-2024-11-17-01-24-10-194
INFO:sagemaker:Creating endpoint-config with name linearregression-endpoint
INFO:sagemaker:Creating endpoint with name linearregression-endpoint


-----!

array([ -0.36231549,  52.23263079, 104.82757707, 157.42252335])

#### 7. Deleting the Endpoint

In [18]:
sklearn_predictor.delete_endpoint(True)

INFO:sagemaker:Deleting endpoint configuration with name: linearregression-endpoint
INFO:sagemaker:Deleting endpoint with name: linearregression-endpoint
